In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
harmony_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\Harmony_data.csv')
IHS_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\IHS_data.csv')
train_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\production_data_train.csv')
test_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\production_data_test.csv')
test_APIs = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\Test_APIs.csv')
test_APIs.shape, harmony_data.shape, IHS_data.shape, train_data.shape, test_data.shape,

((1134, 1), (8930, 7), (9224, 14), (394074, 10), (6489, 11))

## Exploratory Data Analysis

In [3]:
harmony_data.head(3)

,API,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT,GOR_30,GOR_60,GOR_90
0,5123371740000,9056,1025.8680,790.7205,2300.919963,2290.231375,2171.635752
1,5123377360000,4101,NaN,NaN,3786.669639,3838.096579,4012.142621
2,5123378210000,4161,916.0801,839.9280,2399.490505,2291.864862,2472.275585


In [4]:
IHS_data.head(2)

,API,PermitDate,SpudDate,CompletionDate,FirstProductionDate,operatorNameIHS,formation,BasinName,StateName,CountyName,LatWGS84,LonWGS84,BottomHoleLatitude,BottomHoleLongitude
0,33105037530000,2014-09-10,2014-12-31,2015-05-05,2015-03-01 00:00:00.000,CONTINENTAL RESOURCES INCORPORATED,BAKKEN,WILLISTON BASIN,NORTH DAKOTA,WILLIAMS,48.227596,-103.299600,48.2546,-103.29997
1,33105032020000,2013-09-09,2013-11-02,2014-04-28,2014-04-01 00:00:00.000,CONTINENTAL RESOURCES INCORPORATED,BAKKEN,WILLISTON BASIN,NORTH DAKOTA,WILLIAMS,48.230954,-103.189754,48.1983,-103.19195


In [5]:
train_data.head(2)

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate
0,33053063590000,2015,3,17186,26624,1.549168,8213,0.477889,16,2016-08-11 14:03:14.000
1,33053063590000,2015,4,18796,21796,1.159608,7255,0.385986,30,2016-08-11 14:03:14.000


In [6]:
test_data.head(5)

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate,date
0,33053052530100,2014,7,19518,34414,1.763193,10786,0.552618,31,2016-04-06 13:48:08.990,2014-07-01
1,33053052530100,2014,8,18632,32250,1.730893,9959,0.534511,31,2016-04-06 13:48:08.990,2014-08-01
2,33053052530100,2014,9,12700,32768,2.580157,7528,0.592756,30,2016-04-06 13:48:08.990,2014-09-01
3,33053061840000,2015,1,24342,27534,1.131131,5752,0.236299,28,2017-02-23 14:01:12.400,2015-01-01
4,33053061840000,2015,2,12766,18118,1.419239,4143,0.324534,28,2016-04-06 13:48:09.583,2015-02-01


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394074 entries, 0 to 394073
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   API           394074 non-null  int64  
 1   Year          394074 non-null  int64  
 2   Month         394074 non-null  int64  
 3   Liquid        394074 non-null  int64  
 4   Gas           394074 non-null  int64  
 5   RatioGasOil   394074 non-null  float64
 6   Water         394074 non-null  int64  
 7   PercentWater  394074 non-null  float64
 8   DaysOn        394074 non-null  int64  
 9   _LastUpdate   394074 non-null  object 
dtypes: float64(2), int64(7), object(1)
memory usage: 30.1+ MB


#### Checking null values

In [8]:
# Check for missing values
train_data.isnull().sum()

API             0
Year            0
Month           0
Liquid          0
Gas             0
RatioGasOil     0
Water           0
PercentWater    0
DaysOn          0
_LastUpdate     0
dtype: int64

In [9]:
IHS_data.isnull().sum()

API                      0
PermitDate               0
SpudDate                 0
CompletionDate           0
FirstProductionDate    183
operatorNameIHS          0
formation                0
BasinName                0
StateName                0
CountyName               0
LatWGS84                 0
LonWGS84                 0
BottomHoleLatitude       0
BottomHoleLongitude      0
dtype: int64

In [10]:
harmony_data.isnull().sum()

API                       0
LATERAL_LENGTH_BLEND      0
PROP_PER_FOOT           460
WATER_PER_FOOT          633
GOR_30                  135
GOR_60                  135
GOR_90                  135
dtype: int64

### Compare the number of wells

In [11]:
# IHS_data Number of wells
IHS_data.API.unique().shape

(8930,)

In [12]:
# Harmony_data Number of wells
harmony_data.API.unique().shape

(8930,)

In [13]:
# train_data Number of wells
train_data.API.unique().shape[0] + test_data.API.unique().shape[0]

8973

In [14]:
# test_data Number of wells
test_data.API.unique().shape

(1129,)

In [15]:
# Get repetead wells in train and test data
repetead_wells = []
for well in train_data.API.unique():
    if well in test_data.API.unique():
        repetead_wells.append(well)

print(len(repetead_wells))

43


#### Check values of each column

In [ ]:
# Compare the same WELL in train and test data
train_data[(train_data['API'] == 42301322730000) & (train_data['Year'] == 2015)]

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate
393204,42301322730000,2015,2,10801,36697,3.397556,31584,2.924174,0,2018-11-04 10:19:16.577
393205,42301322730000,2015,3,17497,78231,4.471109,67331,3.848145,0,2018-11-04 10:19:16.577
393206,42301322730000,2015,4,12283,65415,5.325653,56301,4.583652,0,2018-11-04 10:19:16.577
393207,42301322730000,2015,5,9661,64795,6.706863,55767,5.772384,0,2018-11-04 10:19:16.577
393208,42301322730000,2015,6,8056,53236,6.608242,45819,5.687562,0,2018-11-04 10:19:16.577
393209,42301322730000,2015,7,7166,53470,7.461624,46020,6.421993,0,2018-11-04 10:19:16.577
393210,42301322730000,2015,8,6502,76269,11.730083,65642,10.095663,0,2018-11-04 10:19:16.577
393211,42301322730000,2015,9,5658,53272,9.415341,45850,8.103570,0,2018-11-04 10:19:16.577
393212,42301322730000,2015,10,5395,51993,9.637257,44749,8.294532,0,2018-11-04 10:19:16.577
393213,42301322730000,2015,11,4885,45718,9.358854,39348,8.054862,0,2018-11-04 10:19:16.577


In [ ]:
# Compare the same WELL in train and test data
test_data[(test_data['API'] == 42301322730000)]

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate,date
6478,42301322730000,2015,2,10801,36697,3.397556,31584,2.924174,0,2018-11-04 10:19:16.577,2015-02-01
6479,42301322730000,2015,3,17497,78231,4.471109,67331,3.848145,0,2018-11-04 10:19:16.577,2015-03-01
6480,42301322730000,2015,4,12283,65415,5.325653,56301,4.583652,0,2018-11-04 10:19:16.577,2015-04-01
6481,42301322730000,2015,5,9661,64795,6.706863,55767,5.772384,0,2018-11-04 10:19:16.577,2015-05-01


In [17]:
# Check if all the wells in test data are in harmony_data and IHS_data
set(test_APIs.index.unique()) - set(harmony_data.index.unique()), set(test_APIs.index.unique()) - set(IHS_data.index.unique())

(set(), set())

In [47]:
# IHS data has more observations than harmony data and wells, it is necessary to verify data that are duplicated 
IHS_data[IHS_data.API.isin(list(IHS_data.API[IHS_data.API.duplicated()]))].head(6)

,API,PermitDate,SpudDate,CompletionDate,FirstProductionDate,operatorNameIHS,formation,BasinName,StateName,CountyName,LatWGS84,LonWGS84,BottomHoleLatitude,BottomHoleLongitude
272,42301320940000,2014-01-02,2014-02-19,2014-04-17,NaN,WPX ENERGY PRODUCTION LLC,BONE SPRING,PERMIAN BASIN,TEXAS,LOVING,31.942399,-103.870208,31.95571,-103.86969
281,42301322750000,2014-08-09,2015-04-08,2015-07-21,NaN,EOG RESOURCES INCORPORATED,BONE SPRING,PERMIAN BASIN,TEXAS,LOVING,31.971822,-103.626594,31.95668,-103.62589
289,42301323710000,2014-10-10,2014-12-18,2015-05-27,2016-01-01 00:00:00.000,APACHE CORPORATION,BONE SPRING,PERMIAN BASIN,TEXAS,LOVING,31.751439,-103.614968,31.73676,-103.63004
290,42301323710000,2014-10-10,2014-12-18,2015-05-27,2015-05-01 00:00:00.000,APACHE CORPORATION,BONE SPRING,PERMIAN BASIN,TEXAS,LOVING,31.751439,-103.614968,31.73676,-103.63004
310,42301326020000,2015-05-21,2015-06-20,2015-08-05,NaN,EOG RESOURCES INCORPORATED,BONE SPRING,PERMIAN BASIN,TEXAS,LOVING,31.986463,-103.619420,31.99981,-103.62003
316,42109325790000,2013-05-29,2013-08-10,2014-02-17,NaN,COG OPERATING LIMITED LIABILITY CORP,BONE SPRING,PERMIAN BASIN,TEXAS,CULBERSON,31.986781,-104.033066,31.99947,-104.03214


In [50]:
# Drop duplicated wells in IHS data
IHS_data = IHS_data.API.drop_duplicates()
IHS_data.shape

(8930,)